In [1]:
from main import * 
import MS


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
df = access_google_sheet(google_sheet_url=google_sheets_urls['hluwill'])

/var/folders/w_/gz8jxmc513q7kbnvgfx67d1c0000gn/T/ipykernel_1060/836761987.py:1: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  df = access_google_sheet(google_sheet_url=google_sheets_urls['hluwill'])


In [3]:
df_candidates = df.loc[df['Party']=='']

df_candidates['level_diff'] = df_candidates['Level'] - 250
df_candidates['fd_multiplier'] = df_candidates['level_diff'].apply(MS.level_fd_multiplier)
df_candidates['adjusted_BA'] = df_candidates['BA'] * df_candidates['fd_multiplier']


In [4]:
df_candidates

,Timestamp,Character Name,Level,Class Type,BA,Party,level_diff,fd_multiplier,adjusted_BA
0,10/24/2023 11:29:17,a,250,DPS,2.500000,,0,1.10,2.750000
1,10/24/2023 11:29:41,b,245,DPS,2.700000,,-5,0.88,2.376000
2,10/24/2023 11:30:11,c,260,DPS,1.800000,,10,1.20,2.160000
3,10/24/2023 11:30:34,d,255,DPS,1.700000,,5,1.20,2.040000
4,10/24/2023 11:30:50,e,253,DPS,2.200000,,3,1.16,2.552000
5,10/24/2023 11:31:13,f,245,Support,1.500000,,-5,0.88,1.320000
6,10/24/2023 11:29:17,g,250,DPS,3.408168,,0,1.10,3.748984
7,10/24/2023 11:29:41,h,245,DPS,1.358492,,-5,0.88,1.195473
8,10/24/2023 11:30:11,i,260,DPS,0.917206,,10,1.20,1.100647
9,10/24/2023 11:30:34,j,255,DPS,1.614667,,5,1.20,1.937600


In [29]:
minimum_average_BA = 1
all_dps_BA = 1.5
supports = df_candidates[df_candidates['Class Type']=='Support']
dps = df_candidates[(df_candidates['Class Type']!='Support')&(df_candidates['BA'] >= minimum_average_BA)].sort_values('adjusted_BA', axis=0, ascending=True)
display(supports, dps)

,Timestamp,Character Name,Level,Class Type,BA,Party,level_diff,fd_multiplier,adjusted_BA
5,10/24/2023 11:31:13,f,245,Support,1.500000,,-5,0.88,1.320000
11,10/24/2023 11:31:13,l,245,Support,1.155007,,-5,0.88,1.016406


,Timestamp,Character Name,Level,Class Type,BA,Party,level_diff,fd_multiplier,adjusted_BA
7,10/24/2023 11:29:41,h,245,DPS,1.358492,,-5,0.88,1.195473
10,10/24/2023 11:30:50,k,253,DPS,1.499720,,3,1.16,1.739675
9,10/24/2023 11:30:34,j,255,DPS,1.614667,,5,1.20,1.937600
3,10/24/2023 11:30:34,d,255,DPS,1.700000,,5,1.20,2.040000
2,10/24/2023 11:30:11,c,260,DPS,1.800000,,10,1.20,2.160000
1,10/24/2023 11:29:41,b,245,DPS,2.700000,,-5,0.88,2.376000
4,10/24/2023 11:30:50,e,253,DPS,2.200000,,3,1.16,2.552000
0,10/24/2023 11:29:17,a,250,DPS,2.500000,,0,1.10,2.750000
6,10/24/2023 11:29:17,g,250,DPS,3.408168,,0,1.10,3.748984


In [30]:
#one support per party
def build_party_support(support_name: str):
    party_curr = [support_name]
    support_damage = supports.loc[supports['Character Name'] == support_name, 'adjusted_BA'].values[0]
    required_BA = minimum_average_BA * 6 - support_damage
    while required_BA > 0:
        dpser = dps.loc[dps.index[0], 'Character Name']
        dpser_ba = dps.loc[dps.index[0], 'adjusted_BA']
        party_curr.append(dpser)
        required_BA -= dpser_ba
        dps.drop(index=dps.index[0], axis=0, inplace=True)

    return party_curr


# Loop through all the supports and add party members until we run out. 
# After we're out of supports, run through remaining dps and see if we can make groups of 6. 
# Turn this into a function
results = []
for support in supports['Character Name']:
    pt = build_party_support(support)
    results.append(pt)

tentative_pt = []
requirement_current = 0
while not dps.empty:
    dpser = dps.loc[dps.index[0], 'Character Name']
    dpser_ba = dps.loc[dps.index[0], 'adjusted_BA']
    tentative_pt.append(dpser)
    requirement_current += dpser_ba
    if requirement_current >= all_dps_BA*6:
        results.append(tentative_pt)
        tentative_pt = []
    dps.drop(index=dps.index[0], axis=0, inplace=True)



    
        



[['f', 'h', 'k', 'j'], ['l', 'd', 'c', 'b'], ['e', 'a', 'g']]

In [35]:
df_candidates.loc[df_candidates['Character Name'].isin(results[2]), 'adjusted_BA'].sum()

9.050984473300002

In [25]:
all_dps_BA = 2

